# Data lab 4 - Portfolio Performance Attribution and Factor Model

#### Note 1: Review what you have learned in Data labs, DataCamp assignments, and in-class sample code.
#### Note 2: Chapter "Performance Attribution" of DataCamp course "Introduction to Portfolio Analysis in Python"  is a useful reference.
#### Note 3: This note serves as a guide. You are free to tinker with it! 

### 1. Revisit the all-weather portfolio you crafted. Create the maximum Sharpe portfolio's daily return dataframe and then merge it with Fama French's five return factors.

#### If you have attempted the optional bonus, why not include the portfolios with L2 regularization and Black-Litterman model too.

In [1]:
# Import the necessary packages
import yfinance as yf
import datetime as dt 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Pick a list of stocks that form your all-weather portfolio
# Draw data for the past five years

#Illustrative, change into yours!


In [3]:
# Keep only the adjusted close in the dataframe
# Note that the date is in the index


In [4]:
# Calculate return using method pct_change
# Find out more about .pct_change with help! 


In [5]:
# function that takes portfolio weights and creates a time-series of daily portfolio returns
def portfolio_return_series(daily_returns, weights):
    
    ''' 
    INPUTS
    daily returns: dataframe of daily returns. Each ticker column contains the series of daily returns for the ticker
    weights: numpy array of the portfolio weight on each ticker (sorted in ascending order)
    
    OUTPUTS
    portfolio_daily_returns: the portfolio return series given the weights
    
    '''    

    # Create portfolio daily returns

    return portfolio_daily_returns

In [6]:
## Install PyPortfolioOpt package
# !pip install PyPortfolioOpt

# Import the packages 


The Maximum Sharpe Portfolio

In [7]:
# Calculate expected returns mu 


# Calculate the covariance matrix S


# Obtain the efficient frontier


# Calculate weights for the maximum Sharpe ratio portfolio


The minimum volatility portfolio

In [8]:
# Obtain the efficient frontier

# Calculate weights for the minimum volatility portfolio


In [9]:
# 1. Daily portfolio returns for the equally-weighted portfolio

# Extract the first element from the function output for daily returns

# 2. Daily portfolio returns for the maximum Sharpe portfolio
# Extract the first element from the function output for daily returns

# 3. Daily portfolio returns for the minimum volatility portfolio
# Extract the first element from the function output for daily returns

# Merge the six series side-by-side into a dataframe
# Note the index is date

# Rename column names


In [10]:
# Inspect the last five observations of the portfolio_returns dataframe


In [11]:
# Read the csv file with factor returns with pd.read_csv()
# Note: place the file where your Jupyter notebook is


# Divide all factor returns by 100
# Consistent with how we calculate portfolio returns


This file is from http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html#Research and is created by CMPT_ME_BEME_OP_INV_RETS_DAILY using the 202208 CRSP database. The 1-month TBill return is from Ibbotson and Associates. For more details on the factor returns, please read http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5_factors_2x3.html

In [12]:
# Inspect the dataframe with info()


In [13]:
# Inspect the last five observations


In [14]:
# Merge portfolio daily and cumulative returns with factor returns


In [15]:
# Inspect the last five observations


In [16]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
portfolio_return = ['portfolio_ew', 'portfolio_maxsharpe', 'portfolio_minvol']

# Print correlation table. Hint use .corr()


### 2. Examine visually the correlation between portfolio and factor returns

#### Hint: Write functions for repetitive lines of codes 

In [17]:
# function that takes portfolio and factor returns and creates 20-day rolling correlation with a return factor
def portfolio_factor_correlation(portfolio_type, df):
    
    ''' 
    INPUTS
    portfolio type: portfolio_ew, portfolio_maxsharpe, portfolio_minvol
    df: dataframe containing portfolio and factor returns
    
    OUTPUTS
    plot the 20-day rolling correlation between a portfolio and a factor return
    
    '''    
    
    for factor in factors:
        
        # Calculate 20-day rolling correlation with the market


        # Plot the correlation between a portfolio and factor returns


    return 

IndentationError: expected an indented block (4268343362.py, line 22)

In [ ]:
# Call the function portfolio_factor_correlation for the equally weighted portfolio


In [ ]:
# Call the function portfolio_factor_correlation for the maximum Sharpe portfolio


In [ ]:
# Call the function portfolio_factor_correlation for the minimum volatility portfolio


### 3. Regress the portfolio return on each factor and assess the portfolio's sensitivity to each factor.

#### For the curious, optional challenge, how do you test whether the intercept (i.e., alpha) is significantly different from the risk-free rate for a single-factor regression?
#### Hint: Write functions for repetitive lines of codes

In [ ]:
# Import the ols function
import statsmodels.api as sm

# function that takes portfolio and factor returns and run a regression of portfolio return on a factor return
# it reports the portfolio sensitivity to a return factor
def portfolio_factor_sensitivity(portfolio_type, df):
    
    ''' 
    INPUTS
    portfolio type: portfolio_ew, portfolio_maxsharpe, portfolio_minvol
    df: dataframe containing portfolio and factor returns
    
    OUTPUTS
    regression result of a portfolio on a return factor
    
    '''    
    for factor in factors:
        
        # Create the regression model object

        # Fit the model

        # Print the parameters of the fitted model

        # Optional challenge: testing the hypothesis that the intercept is significantly different from the risk-free rate
        # Hint: F-test

        
    return 

In [ ]:
# Call the function portfolio_factor_sensitivity for the equally weighted portfolio


In [ ]:
# Call the function portfolio_factor_sensitivity for the maximum Sharpe portfolio


In [ ]:
# Call the function portfolio_factor_sensitivity for minimum volatility portfolio


### 4. Regress the portfolio return on all factors and assess the portfolio's sensitivity to factors.

#### For the curious, optional challenge, how do you test whether the intercept (i.e., alpha) is significantly different from the risk-free rate for a multi-factor regression?
#### Hint: Write functions for repetitive lines of codes

In [ ]:
# Import the ols function
import statsmodels.api as sm

# function that takes portfolio and factor returns and run a regression of portfolio return on a return factor
# it reports the portfolio sensitivity to a return factor
def portfolio_all_factor_sensitivity(portfolio_type, df):
    
    ''' 
    INPUTS
    portfolio type: portfolio_ew, portfolio_maxsharpe, portfolio_minvol
    df: dataframe containing portfolio and factor returns
    
    OUTPUTS
    regression result of a portfolio on a return factor
    
    '''    
        
    # Create the model object

    
    # Fit the model

    # Print the parameters of the fitted model
    
    # Optional challenge: testing the hypothesis that the intercept is significantly different from the risk-free rate
    # Hint: F-test


    return 

In [ ]:
# Call the function portfolio_all_factor_sensitivity for the equally weighted portfolio


In [ ]:
# Call the function portfolio_all_factor_sensitivity for the maximum Sharpe portfolio


In [ ]:
# Call the function portfolio_all_factor_sensitivity for the minimum volatility portfolio


### 5. Optional Bonus. Construct a multi-factor pricing model for assets based on Arbitrage Pricing Theory.

#### The Arbitrage Pricing Theory (APT) is a theory of asset pricing that holds that an asset’s returns can be forecasted with the linear relationship of an asset’s expected returns and the macroeconomic (e.g., GDP, changes in inflation, yield curve changes, changes in interest rates, market sentiments, exchange rates) or firm-specific statistical factors that affect the asset’s risk. Hint: You can draw these variables straight into your Jupyter notebook via Refinitiv API.

#### The APT is a substitute for the Capital Asset Pricing Model (CAPM) in that both assert a linear relation between assets’ expected returns and their covariance with other random variables. (In the CAPM, the covariance is with the market portfolio’s return.) The covariance is interpreted as a measure of risk that investors cannot avoid by diversification. The slope coefficient in the linear relation between the expected returns and the covariance is interpreted as a risk premium ~ "Arbitrage Pricing Theory (Guberman and Wang 2005).

Acknowledgement: This notebook is inspired by DataCamp course "Introduction to Portfolio Analysis in Python" by Charlotte Werger.